In [4]:
# import modules

import numpy as np
import pandas as pd
pd.set_option('display.float_format', '{:_.0f}'.format)
from scipy.stats import poisson, norm
from math import factorial, exp, log
import datetime, time
from decimal import *

import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
import matplotlib as mpl
%matplotlib inline
mpl.rcParams["axes.formatter.min_exponent"] = 20  # no scientific notation in graphs
plt.rcParams['axes.titley'] = 1.0    # y is in axes-relative coordinates.
plt.rcParams['axes.titlepad'] = 10   # pad is in points... default is 6
import seaborn as sns
sns.set_theme()

In [160]:
# CONSTANTS treated as variables for simulation

# inputs to the Drake equation
default_params = dict(
    # time step increments for calculations
    YEAR_STEPS = 1_000_000,
    # galactic attributes
    NUM_GALAXY = 100_000_000_000,  # number of stars in a galaxy
    GALAXY_RADIUS_LYR = 50_000,  # radius of the galaxy in lightyears
    MODERN_ERA = 10_000_000_000,  # years since 2nd gen stars, order of magnitude approx
    # pseudo-constants (treated as constants until Monte Carlo simulation)
    RS = 2, 
    FP = 1, 
    NE = 0.4, 
    FL = 1, 
    FI = 1, 
    FC = 0.1, 
    L = 1000,
    YEARS_PLANETS_TO_HABITABLE = 2_000_000_000, 
    YEARS_HABITABLE_TO_LIFE = 1_000_000_000, 
    YEARS_LIFE_TO_COMPLEX_LIFE = 1_000_000_000, 
    YEARS_COMPLEX_TO_INTELLIGENCE = 200_000_000, 
    YEARS_INTELLIGENCE_TO_CULTURE = 20_000_000, 
    YEARS_CULTURE_TO_TECH = 200_000, 
    EXTINCTION_SIMPLE = 5_000_000_000, 
    EXTINCTION_COMPLEX = 2_000_000_000, 
    EXTINCTION_INTELLIGENT = 200_000_000, 
    EXTINCTION_CULTURAL = 1_000_000, 
    # EXTINCTION_TECHNOLOGICAL = L,
    WEIBULL_SHAPE_PARAMETER = 0.4,  # k
    WEIBULL_SCALE_PARAMETER = 20_000  # lambda    
)

random_params = dict(
    # time step increments for calculations
    YEAR_STEPS = [1_000_000],
    # galactic attributes
    NUM_GALAXY = [100_000_000_000],  # number of stars in a galaxy
    GALAXY_RADIUS_LYR = [50_000],  # radius of the galaxy in lightyears
    MODERN_ERA = [10_000_000_000],  # years since 2nd gen stars, order of magnitude approx
    # pseudo-constants (treated as constants until Monte Carlo simulation)
    RS = [1, 2, 3],
    FP = [0.05, 0.2, 0.5, 1.0, 2.0],
    NE = [0.02, 0.1, 0.4, 1.0],
    FL = [0.01, 0.1, 1],
    FI = [0.01, 0.1, 1],
    FC = [0.01, 0.1, 1],
    L = [100, 1_000, 10_000, 100_000, 1_000_000],
    YEARS_PLANETS_TO_HABITABLE = [1_000_000_000, 2_000_000_000, 3_000_000_000], 
    YEARS_HABITABLE_TO_LIFE = [100_000_000, 1_000_000_000, 5_000_000_000], 
    YEARS_LIFE_TO_COMPLEX_LIFE = [100_000_000, 1_000_000_000, 5_000_000_000, 10_000_000_000], 
    YEARS_COMPLEX_TO_INTELLIGENCE = [100_000, 1_000_000, 50_000_000, 200_000_000, 1_000_000_000], 
    YEARS_INTELLIGENCE_TO_CULTURE = [10_000, 100_000, 2_000_000, 20_000_000, 200_000_000], 
    YEARS_CULTURE_TO_TECH = [1_000, 10_000, 100_000, 200_000, 2_000_000], 
    EXTINCTION_SIMPLE = [200_000_000, 1_000_000_000, 2_000_000_000, 5_000_000_000],  # these are mass extinction events
    EXTINCTION_COMPLEX = [200_000_000, 1_000_000_000, 2_000_000_000], # these are mass extinction events
    EXTINCTION_INTELLIGENT = [10_000, 100_000, 1_000_000, 10_000_000, 200_000_000, 1_000_000_000], 
    EXTINCTION_CULTURAL = [1_000, 10_000, 100_000, 1_000_000, 10_000_000], 
    # EXTINCTION_TECHNOLOGICAL = L,
    WEIBULL_SHAPE_PARAMETER = [0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0],  # k
    WEIBULL_SCALE_PARAMETER = [100, 250, 500, 750, 1000, 5_000, 10_000, 20_000, 50_000, 100_000]  # lambda
)


In [161]:
params = default_params.copy()

In [163]:
def star_lifeforms(params):
    
    """
    doesn't account for mass extinctions, star lifespan, etc
    if everything goes perfectly, this is how long everything would take
    could use any distribution, we are going to use clt to get a normal
    """
    
    sd_low = 0.01
    sd_med = 0.05
    sd_high = 0.20
    
    y_h = params['YEARS_PLANETS_TO_HABITABLE']
    y_l = params['YEARS_HABITABLE_TO_LIFE']
    y_cx = params['YEARS_LIFE_TO_COMPLEX_LIFE']
    y_i = params['YEARS_COMPLEX_TO_INTELLIGENCE']
    y_cl = params['YEARS_INTELLIGENCE_TO_CULTURE']
    y_t = params['YEARS_CULTURE_TO_TECH']
    x_t = params['L']
    k = 0.5  # params['WEIBULL_SHAPE_PARAMETER']
    l = params['L']  # params['WEIBULL_SCALE_PARAMETER'] or params['L'] (different)
        
    # years to habitable
    t_h = norm.rvs(loc=y_h, scale=y_h*sd_high)
    
    # years to life
    t_l = norm.rvs(loc=y_l, scale=y_l*sd_high)
    
    # years to complex life
    t_cx = norm.rvs(loc=y_cx, scale=y_cx*sd_high)
    
    # years to intelligence
    t_i = norm.rvs(loc=y_i, scale=y_i*sd_high)
    
    # years to culture
    t_cl = norm.rvs(loc=y_cl, scale=y_cl*sd_high)
    
    # years to technology
    t_t = norm.rvs(loc=y_t, scale=y_t*sd_high)
    
    # years to extinction
    # # normal
    # t_x = norm.rvs(loc=x_t, scale=x_t*sd_high)
    # random weibull number
    t_x = int(np.random.weibull(k) * l)
        
    return [int(t_h), int(t_l), int(t_cx), int(t_i), int(t_cl), int(t_t), int(t_x)]

In [189]:
# birth of intelligent life
temp = star_lifeforms(params)
print(temp)
print(f"{sum(temp[:-1]):_.0f}")

# life of species
print(f"{temp[-1]:_.0f}")

# extinction of life
print(f"{sum(temp):_.0f}")

[2591965349, 707029966, 1164760631, 232472687, 19482187, 196413, 96]
4_715_907_233
96
4_715_907_329


In [226]:
def star_habitable_lifespan():
    """
    combines habitability percentages from 
        'star lifespan, percent, habitability.xlsx'
    with stellar lifespans and creates a livible lifespan for a star
    """
    temp_rand = np.random.random()
    star_lifespan = 0
    
    if temp_rand < 0.001:
        star_lifespan = 1_000_000_000
    elif temp_rand < 0.016:
        star_lifespan = 5_000_000_000
    elif temp_rand < 0.054:
        star_lifespan = 10_000_000_000
    elif temp_rand < 0.078:
        star_lifespan = 50_000_000_000
    elif temp_rand < 0.155:
        star_lifespan = 100_000_000_000
        
    # add some variability with normal distributions
    return int(norm.rvs(loc=star_lifespan, scale=star_lifespan*0.2))
    

In [265]:
star_habitable_lifespan()

0

In [306]:
# TODO

def lifeform_extinctions(params):
    
    """
    star extinctions
    mass extinctions
    cataclysms
    rouge ai paperclip monsters
    does not include tech extinctions which are included in the lifespan calcs above
    """
    
    sd_low = 0.01
    sd_med = 0.05
    sd_high = 0.15
    
    x_s = params['EXTINCTION_SIMPLE']
    x_c = params['EXTINCTION_COMPLEX']
    x_i = params['EXTINCTION_INTELLIGENT']
    x_cl = params['EXTINCTION_CULTURAL']
    
    y_h = params['YEARS_PLANETS_TO_HABITABLE']
    y_l = params['YEARS_HABITABLE_TO_LIFE']
    y_cx = params['YEARS_LIFE_TO_COMPLEX_LIFE']
    y_i = params['YEARS_COMPLEX_TO_INTELLIGENCE']
    y_cl = params['YEARS_INTELLIGENCE_TO_CULTURE']
        
    # mass extinctions
    temp_yrs = y_h + x_s
    x_year_1 = norm.rvs(loc=temp_yrs, scale=temp_yrs*sd_high)  # simple
    
    temp_yrs = y_h + y_l + x_c
    x_year_2 = norm.rvs(loc=temp_yrs, scale=temp_yrs*sd_high)  # complex
    
    temp_yrs = y_h + y_l + y_cx + x_i
    x_year_3 = norm.rvs(loc=temp_yrs, scale=temp_yrs*sd_high)  # intelligent
    
    temp_yrs = y_h + y_l + y_cx + y_cl + x_cl
    x_year_4 = norm.rvs(loc=temp_yrs, scale=temp_yrs*sd_high)  # cultural
    
    return int(min(x_year_1, x_year_2, x_year_3, x_year_4))


In [324]:
lifeform_extinctions(params)

4151591523

In [162]:
# number of stars

def star_birth(params, current_year):
    """
    star birth
    currently 30 times lower than at the start of the universe
    peaked 8 billion years ago
    approx rate calc assumes 30x at 10B ago, and Rs from now forward, linear interpolation
    
    star death
    not based on astro physics, just used to balance the births vs death at num_galaxy
    """
    # star birth
    rate = params['RS'] * max(((params['MODERN_ERA'] - current_year) * 30) / params['MODERN_ERA'], 1)
    
    return rate * params['YEAR_STEPS']


def star_death(params, current_year, num_stars):
    """
    star birth
    currently 30 times lower than at the start of the universe
    peaked 8 billion years ago
    approx rate calc assumes 30x at 10B ago, and Rs from now forward, linear interpolation
    
    star death
    not based on astro physics, just used to balance the births vs death at num_galaxy
    """
    # star birth
    rate = params['RS'] * max(((params['MODERN_ERA'] - current_year) * 30) / params['MODERN_ERA'], 1)
    
    return rate * params['YEAR_STEPS'] * num_stars / params['NUM_GALAXY']


# number of new planets
def new_planets(params, num_new_stars):
    """
    New stars create new planets. These planets are not habitable yet, but 
    they only count if they will become habitable, and will develop life.
    
    Incorporates all of the following Drake parameters:
    fp = fraction of stars that have planets
    ne = mean number of planets that could support life per star with planets
    fl = fraction of life-supporting planets that develop life
    """
    return num_new_stars * params['FP'] * params['NE'] * params['FL']
